In [2]:

import matplotlib.pyplot as plt
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import brewer2mpl
import re
import pandas as pd
import numpy as np
from collections import defaultdict


init_notebook_mode(connected=True)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

df = pd.read_csv('movies_list.csv') #скачиваем подготовленные данные
df.head()

,Unnamed: 0,countries,genres,movie_duration,year
0,0,['США'],"['мультфильм', 'комедия', 'приключения', 'семе...",89,2008
1,1,['Россия'],"['драма', 'история']",124,2008
2,2,"['США', 'Германия', 'Китай']","['ужасы', 'фэнтези', 'боевик', 'триллер', 'при...",112,2008
3,3,['Россия'],['комедия'],106,2008
4,4,"['США', 'Германия']","['фэнтези', 'боевик', 'триллер', 'криминал']",110,2008


In [16]:
def parse_list(lst_str):
    return list(map(lambda x: x.strip(), re.sub(r'[\[\]]', '', lst_str).split(',')))

#determine the unique genres
genres = set()
for m in df.genres:
    genres.update(g for g in parse_list(m))
genres = sorted(genres)

for genre in genres:
    df[genre] = [genre in movie for movie in df.genres]
#df.groupby(['year'])[genres].sum().idxmax(axis=1)

df_genres = df.groupby(['year'])[genres].sum()
temp = pd.DataFrame(df_genres)
order = np.argsort(-temp.values, axis=1)[:, :3]
result = pd.DataFrame(temp.columns[order], 
                      columns=['{}'.format(i) for i in range(1, 4)],
                      index=temp.index)

pd.DataFrame(result)


,1,2,3
year,,,
2008,'приключения','боевик','комедия'
2009,'приключения','боевик','драма'
2010,'приключения','фэнтези','боевик'
2011,'приключения','боевик','комедия'
2012,'приключения','боевик','фантастика'
2013,'приключения','боевик','фантастика'
2014,'приключения','боевик','фантастика'
2015,'приключения','боевик','фантастика'
2016,'приключения','фэнтези','семейный'


In [2]:
def parse_list(lst_str):
    return list(map(lambda x: x.strip(), re.sub(r'[\[\]]', '', lst_str).split(',')))

df['genres'] = df['genres'].fillna('[]')
genres_data = []
for record in df.to_dict(orient = 'records'):
    genres_lst = parse_list(record['genres'])
    for genre in genres_lst:
        copy = record.copy()
        copy['genre'] = genre
        copy['weight'] = 1./len(genres_lst)
        genres_data.append(copy)

genres_df = pd.DataFrame.from_dict(genres_data)
genres_coincidents = {}

for item in df.genres:
    parsed_genres = parse_list(item)
    for genre1 in parsed_genres:
        if genre1 not in genres_coincidents:
            genres_coincidents[genre1] = defaultdict(int)
        for genre2 in parsed_genres:
            genres_coincidents[genre1][genre2] += 1

genres_coincidents_df = pd.DataFrame.from_dict(genres_coincidents).fillna(0)

# отнормируем таблицу на количество фильмов каждого жанра
genres_coincidents_df_norm = genres_coincidents_df\
    .apply(lambda x: x/genres_df.groupby('genre').year.count(), axis = 1)

heatmap = go.Heatmap(
    z = genres_coincidents_df_norm.values,
    x = genres_coincidents_df_norm.index.values,
    y = genres_coincidents_df_norm.columns
)
layout = go.Layout(
    title = 'Связанные жанры'
)

fig = go.Figure(data = [heatmap], layout = layout)
iplot(fig)

In [21]:
df.groupby(['year'])['movie_duration'].describe(percentiles=[])

,count,mean,std,min,50%,max
year,,,,,,
2008,20.0,108.15,16.352289,88.0,105.0,150.0
2009,20.0,117.25,25.047061,80.0,114.0,162.0
2010,20.0,107.05,18.880441,75.0,102.5,148.0
2011,20.0,111.05,19.677599,85.0,111.0,154.0
2012,20.0,118.00,29.311395,65.0,110.5,172.0
2013,20.0,119.30,18.510594,90.0,121.0,161.0
2014,20.0,119.40,23.473165,89.0,116.0,169.0
2015,20.0,117.05,23.712033,74.0,120.5,150.0
2016,20.0,115.95,19.765667,87.0,115.5,156.0


In [18]:
def parse_list(lst_str):
    return list(map(lambda x: x.strip(), re.sub(r'[\[\]]', '', lst_str).split(',')))

#determine the unique genres
countries = set()
for m in df.countries:
    countries.update(g for g in parse_list(m))
countries = sorted(countries)

for country in countries:
    df[country] = [country in movie for movie in df.countries]
#df.groupby(['year'])[countries].sum().idxmax(axis=1)
df_countries = df.groupby(['year'])[countries].sum()
temp = pd.DataFrame(df_countries)
order = np.argsort(-temp.values, axis=1)[:, :3]
result = pd.DataFrame(temp.columns[order], 
                      columns=['{}'.format(i) for i in range(1, 4)],
                      index=temp.index)

pd.DataFrame(result)


,1,2,3
year,,,
2008,'США','Россия','Германия'
2009,'США','Россия','Великобритания'
2010,'США','Россия','Великобритания'
2011,'США','Россия','Великобритания'
2012,'США','Великобритания','Германия'
2013,'США','Россия','Франция'
2014,'США','Великобритания','Канада'
2015,'США','Великобритания','Россия'
2016,'США','Великобритания','Россия'


In [10]:
country_count = df[countries].sum().sort_values(ascending = False).head(5)
pd.DataFrame({'Country Count': country_count})

,Country Count
'США',165
'Россия',33
'Великобритания',31
'Канада',12
'Германия',12


In [9]:

genre_count = df[genres].sum().sort_values(ascending = False).head(5)
pd.DataFrame({'Genre Count': genre_count})

Топ-5 жанров


,Genre Count
'приключения',133
'боевик',100
'комедия',78
'фэнтези',73
'фантастика',73


In [45]:
def parse_list(lst_str):
    return list(map(lambda x: x.strip(), re.sub(r'[\[\]]', '', lst_str).split(',')))

#determine the unique genres
countries = set()
for m in df.countries:
    countries.update(g for g in parse_list(m))
countries = sorted(countries)
for country in countries:
    df[country] = [country in movie for movie in df.countries]
#df_countries = df.groupby(['year'])["'Россия'"].sum()
#pd.DataFrame(df_countries)
#rus_year_df = df.groupby(['year'])["'Россия'"].sum()
rus_usa_year_df = pd.DataFrame(df.groupby(['year'],as_index = False)["'Россия'","'США'"].sum())
print(rus_usa_year_df.columns)
rus_usa_year_df.rename(columns={"'США'": 'Usa', "'Россия'": 'Rus'}, inplace=True)
print(rus_usa_year_df.columns)
trace_rus = go.Scatter(
    x = rus_usa_year_df.year,
    y = rus_usa_year_df.Rus,
    mode = 'lines',
    name = u'Россия'
)
trace_usa = go.Scatter(
    x = rus_usa_year_df.year,
    y = rus_usa_year_df.Usa,
    mode = 'lines',
    name = u'США'
)
layout = go.Layout(
    title='Динамика',
)   

fig = go.Figure(data = [trace_rus, trace_usa], layout = layout)
iplot(fig)

Index(['year', ''Россия''], dtype='object')
Index(['year2', 'Rus'], dtype='object')
